In [1]:
import pandas as pd
import numpy as np
from itertools import permutations, combinations
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

Reading all month data files and gross profit data file

In [2]:
month1Data = pd.read_csv('2022-01_line_items.csv')
month2Data = pd.read_csv('2022-02_line_items.csv')
month3Data = pd.read_csv('2022-03_line_items.csv')
month4Data = pd.read_csv('2022-04_line_items.csv')
month5Data = pd.read_csv('2022-05_line_items.csv')
grossProfitData = pd.read_csv('GrossProfits.csv')

We merge all the monthly data into a single dataframe for easier processing. We also remove the NaN values from the Gross Profit excel sheet

In [3]:
data = pd.concat([month1Data, month2Data, month3Data, month4Data, month5Data])
grossProfitData.dropna(inplace = True)

In [4]:
data.drop('Unnamed: 0', axis = 1, inplace = True)

Checking if there are any NaN data in the monthly dataset

In [5]:
data.isna().any()

item         True
qty         False
price       False
date        False
order_id    False
dtype: bool

In [6]:
data

,item,qty,price,date,order_id
0,Cash Sale,1,13.0,14/01/2022,123668
1,Cash Sale,1,3.0,14/01/2022,123669
2,Cash Sale,1,30.5,14/01/2022,123670
3,Cash Sale,1,12.0,14/01/2022,123671
4,Cash Sale,1,35.0,14/01/2022,123673
...,...,...,...,...,...
833,Mexican (Small),1,16.0,2022-05-04,127376
834,Cash Sale,6,346.5,2022-05-04,127377
835,Cash Sale,6,620.5,2022-05-04,127378
836,Charles,1,0.0,2022-05-04,127387


### Data Cleaning

There is a lot of cleaning required in the dataset as we can see from above the price of items are 0 for many rows, item 'Cash Sale' needs to be removed and there is a lot of further cleaning required which is done in below step

In [7]:
# Removing rows with Cash Sale and Unknown data values
data = data[(data['item'] != 'Cash Sale') & (data['item'] != '#NAME?')]

# Removing the rows with price of items as 0
data = data[data['price'] != 0]

# Removing the rows which contains  NaN values in the dataframe
data.dropna(inplace = True)

# Removing the rows with item columns as digits and also cleaning the item names by removing punctuations so that data is clean
data = data[~data['item'].str.isdigit()]
data = data[~data.item.str.startswith('-')]
data.loc[:, 'item'] = data['item'].str.replace(r'[()]',"")

In [8]:
data

,item,qty,price,date,order_id
58,dinne,1,41.5,17/01/2022,123796
234,Gelati Cone,1,5.0,27/01/2022,124241
237,Giandiotto,1,9.0,27/01/2022,124241
292,Can Club & Dry RTD,1,10.0,14/01/2022,123672
293,Chicken & Mushroom Main,1,21.5,14/01/2022,123672
...,...,...,...,...,...
825,Antipasto Small,1,19.5,2022-05-04,127374
827,Chalk Hill Chardonnay,1,9.0,2022-05-04,127376
829,Margherita Small,1,12.5,2022-05-04,127376
833,Mexican Small,1,16.0,2022-05-04,127376


We remove the order_id which have only one item as their order since we want to look for items which are most commonly sold together

In [9]:
data = data[data['order_id'].duplicated(keep = False)]

In [10]:
data

,item,qty,price,date,order_id
234,Gelati Cone,1,5.0,27/01/2022,124241
237,Giandiotto,1,9.0,27/01/2022,124241
292,Can Club & Dry RTD,1,10.0,14/01/2022,123672
293,Chicken & Mushroom Main,1,21.5,14/01/2022,123672
295,Coke Zero,1,4.5,14/01/2022,123672
...,...,...,...,...,...
824,Angove Shiraz Cab,1,8.5,2022-05-04,127374
825,Antipasto Small,1,19.5,2022-05-04,127374
827,Chalk Hill Chardonnay,1,9.0,2022-05-04,127376
829,Margherita Small,1,12.5,2022-05-04,127376


##### **1. What items are most commonly sold together? What is the gross-profit” per dish sold together?**

Grouping the data into a single column with same order_id and we separate the item names through the comma in the column

In [11]:
data['Group'] = data.groupby('order_id')['item'].transform(lambda x: ','.join(x))
data = data[['order_id', 'Group']].drop_duplicates()
data

,order_id,Group
234,124241,"Gelati Cone,Giandiotto"
292,123672,"Can Club & Dry RTD,Chicken & Mushroom Main,Cok..."
305,123719,"Garlic Bread,Tartufo"
309,123674,"Caffe Latte,Lemon Lime Bitters,Potato Small,Sc..."
318,123675,"Capriciosa Small,Garlic Focaccia,Garlic Focacc..."
...,...,...
802,127367,"Aussie,Capriciosa Large,Carlton Draught,Cascad..."
815,127368,"Eureka Main,Herb Focaccia Small,OBrien Pale Ale"
821,127371,"Mushroom Small,Supremo Small,Wine Corkage"
824,127374,"Angove Shiraz Cab,Antipasto Small"


Counting for the combinations which the items have in the dataframe and store them in a new dataframe

In [12]:
count = Counter()
for row in data['Group']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

combinationDf = pd.DataFrame.from_dict(count, orient='index').reset_index().rename(columns={'index':'Combinations', 0:'count'})
combinationDf.sort_values(by = 'count', ascending = False, inplace = True)

Selecting the top 20 combinations of the items so that we can see the items which most commonly sold together

In [13]:
top20Comb = pd.DataFrame(combinationDf['Combinations'].tolist(), index=combinationDf.index).rename(columns=
                                                                                       {0:'item1', 1:'item2'}).iloc[0:20,:]
top20Comb['count'] = combinationDf['count'][0:20]

The above dataframe has three columns; item1, item2, and count. The item1 shows the first item of the combination, item2 shows the second one, and the count shows how much times these items have been sold together over the period of 5 months. Next, finding out the gross profit for each item using the GrossProfit sheet

In [15]:
item1GP = grossProfitData[grossProfitData['Dish'].isin(top20Comb.item1)].rename(columns = {'Dish':'item1'})
item2GP = grossProfitData[grossProfitData['Dish'].isin(top20Comb.item2)].rename(columns = {'Dish':'item2'})

In [16]:
tempDF = top20Comb.merge(item1GP, on = 'item1', how = 'left')[['item1', 'item2',
                                                        'count', 'Gross Profit']].rename(columns = {'Gross Profit':'item1GP'})
GrossProfitDF = tempDF.merge(item2GP, on = 'item2', how = 'left')[['item1', 'item2', 'count',
                                                        'item1GP', 'Gross Profit']].rename(columns = {'Gross Profit':'item2GP'})

In [17]:
GrossProfitDF['combinationGP ($)'] = GrossProfitDF[['item1GP', 'item2GP']].replace({'\$': '', ',': ''},
                                                                               regex=True).astype(float).sum(axis = 1)

In [18]:
GrossProfitDF

,item1,item2,count,item1GP,item2GP,combinationGP ($)
0,Coke,Garlic Bread,68,$4.5,$7,11.50
1,Bolognese Entree,Garlic Bread,46,$13.83,$7,20.83
2,Bowl Of Chips,Garlic Bread,45,$4.53,$7,11.53
3,Garlic Bread,Super Special Large,40,$7,$14.03,21.03
4,Garlic Bread,Lemon Lime Bitters,40,$7,$5,12.00
5,Coke Zero,Garlic Bread,37,$4.5,$7,11.50
6,Bowl Of Chips,Coke,37,$4.53,$4.5,9.03
7,Carbonara Main,Coke,36,$15.76,$4.5,20.26
8,Super Special Small,Wine Corkage,34,$9.50,$7,16.50
9,Super Special Large,Wine Corkage,34,$14.03,$7,21.03


The above dataframe shows the Gross Profit per dish and the combined gross profit as well for the items which are sold together most commonly.

##### **2. What is the percentage breakdown of sales on a per-category basis (for example 20% of sales are gourmet pizzas, 10% of sales normal pizza. 12% of sales are “From the sea”, etc). What is the weight average % gross profit per category**

#### Making Categories using the Menu

In [19]:
pizza = ['Super Special Small', 'Super Special Large', 'Eureka Special Small', 'Eureka Special Large', 'Capriciosa Small',
         'Capriciosa Large', 'Margherita Small', 'Margherita Large', 'Napoletana Small', 'Napoletana Large', 
         'Different Corners Small', 'Different Corners Large', 'Marinara Small', 'Marinara Large', 'Vegetarian Small',
         'Vegetarian Large', 'Aussie Small', 'Aussie Large', 'American Small', 'American Large', 'John Special Small',
         'John Special Large', 'Lucio Special Small', 'Lucio Special Large', 'Tropical Small', 'Tropical Large', 
         'Mushroom Small', 'Mushroom Large', 'Mexican Small', 'Mexican Large', 'Chicken Small', 'Chicken Large', 
         'Pepperoni Small', 'Pepperoni Large', 'Meat Lovers Small', 'Meat Lovers Large']

gourmetPizza = ['Antipasto Small', 'Antipasto Large', 'Avocado Small', 'Avocado Large', 'Pesto Small', 'Pesto Large',
                'Pomodoro Small', 'Pomodoro Large', 'Patate Small', 'Patate Large', 'Supremo Small', 'Supremo Large']

starters = ['Bowl Of Chips', 'Basket of Bread Small', 'Basket of Bread Large', 'Garlic Bread', 
            'Bruschetta Tomato', 'Bruschetta Capsicum', 'Affetato Platter', 'Oysters 1/2 dozen', 'Oysters Dozen',
            'Oysters Kilpatrick 1/2 dozen', 'Oysters Kilpatrick Dozen', 'Garlic Focaccia Small', 'Garlic Focaccia Large']

soups = ['Minestrone', 'Stracciatella']

pasta = ['Arrabiata Entree', 'Arrabiata Main', 'Bolognese Entree', 'Bolognese Main', 'Broccoli Entree', 'Broccoli Main',
         'Carbonara Entree', 'Carbonara Main', 'Chicken & Mushroom Entree', 'Chicken & Mushroom Main', 'Vegetarian Entree',
         'Vegetarian Main', 'Lasagna Entree', 'Lasagna Main', 'Eureka Entree', 'Eureka Main', 'Garlic Prawns Entree',
         'Garlic Prawns Main', 'Marinara Entree', 'Marinara Main', 'Napoli Entree', 'Napoli Main', 'Romana Entree',
         'Romana Main']

risotto = ['Primavera Entree', 'Primavera Main', 'Scallop Entree', 'Scallop Main', 'Chicken & Avocado Entree', 
           'Chicken & Mushroom Main']

fromTheSea = ['Calamari Fritti', 'Flounder', 'Mussels Marinara', 'Garlic Prawn', 'Bugs On Spaghetti']

fromTheLand = ['Parma Chicken Entree', 'Parma Chicken Main', 'Parma Veal Entree', 'Parma Veal Main', 'Schnitzel Chicken Entree',
               'Schnitzel Chicken Main', 'Schnitzel Veal Entree', 'Schnitzel Veal Main', 'Scallopine', 'Vitello Sorrento']

sauces = ['Garlic Butter', 'Diane', 'Mushroom', 'Pepper', 'Red Wine']

steak = ['T-bone 400g', 'Scotch Fillet 300g', 'T-Bone Steak']

salads = ['Side Salad', 'Seafood Salad', 'Warm Beef Salad', 'Caesar Salad', 'Add Chicken', 'Warm Chicken Salad']

pancakes = ['Jamaican Pancake', 'Blueberry Pancake', 'Citrus Pancake', 'Plain Pancake']

sweets = ['Tiramisu', "Chef's Cake"]

gelati = ['Gelati Cone', 'Gelati Coppa']

frozenDesserts = ['Cassata', 'Gianduiotto', 'Mint Ice', 'Tartufo']

softDrinks = ['Coca Cola', 'Lemon Lime Bitters', 'Coke Zero', 'Mineral Water 500Ml', 'Sprite', 'Soda',
              'Coke', 'Coke Bottle', 'Mineral Water 250Ml']

additional = ['Wine Corkage', 'Carlton Draught', 'Great Northern Super Crisp']

hot = ['Cappuccino', 'Caffe Latte', 'Hot Chocolate']

beers = ['Caltron Draught', 'Carlton Dry', 'Cascade Light']

In [20]:
data = pd.concat([month1Data, month2Data, month3Data, month4Data, month5Data])

In [21]:
data.drop('Unnamed: 0', axis = 1, inplace = True)

# Removing rows with Cash Sale and Unknown data values
data = data[(data['item'] != 'Cash Sale') & (data['item'] != '#NAME?')]

# Removing the rows with price of items as 0
data = data[data['price'] != 0]

# Removing the rows which contains  NaN values in the dataframe
data.dropna(inplace = True)

# Removing the rows with item columns as digits and also cleaning the item names by removing punctuations so that data is clean
data = data[~data['item'].str.isdigit()]
data = data[~data.item.str.startswith('-')]
data.loc[:, 'item'] = data['item'].str.replace(r'[()]',"")

In [22]:
conditions = [
    (data['item'].isin(pizza)),
    (data['item'].isin(gourmetPizza)),
    (data['item'].isin(starters)),
    (data['item'].isin(soups)),
    (data['item'].isin(pasta)),
    (data['item'].isin(risotto)),
    (data['item'].isin(fromTheSea)),
    (data['item'].isin(fromTheLand)),
    (data['item'].isin(sauces)),
    (data['item'].isin(steak)),
    (data['item'].isin(salads)),
    (data['item'].isin(pancakes)),
    (data['item'].isin(sweets)),
    (data['item'].isin(gelati)),
    (data['item'].isin(frozenDesserts)),
    (data['item'].isin(softDrinks)),
    (data['item'].isin(additional)),
    (data['item'].isin(hot)),
    (data['item'].isin(beers))
]

values = ['pizza', 'gourmetPizza', 'starters', 'soups', 'pasta', 'risotto', 'fromTheSea', 'fromTheLand', 'sauces', 'steak',
          'salads', 'pancakes', 'sweets', 'gelati', 'frozenDesserts', 'softDrinks', 'additional', 'hot', 'beers']

data['category'] = np.select(conditions, values)
data = data[(data['category'] != '0')]

In [23]:
categoryQty = data.groupby('category').sum().reset_index()[['category', 'qty']]

In [24]:
categoryQty['percentBreakdown'] = round((categoryQty['qty'] / categoryQty['qty'].sum()) * 100, 2)

In [26]:
categoryQty.sort_values(by = 'percentBreakdown', ascending = False)

,category,qty,percentBreakdown
9,pasta,1881,20.96
10,pizza,1818,20.26
13,softDrinks,1121,12.49
0,additional,910,10.14
15,starters,888,9.89
2,fromTheLand,659,7.34
1,beers,267,2.97
7,hot,243,2.71
12,salads,226,2.52
6,gourmetPizza,202,2.25


The above dataframe shows percentage breakdown of sales on a per-category basis. We can see that Pasta, Pizza, Soft Drinks, and Starters are the categories with most number of sales.

**Note** Additional category includes those items which are not listed in the Gross Profit excel sheet (majority of the items are from the Wine, Beers, and Liquors category.

In [27]:
grossProfitData = pd.read_csv('GrossProfits.csv')
grossProfitData.dropna(inplace = True)

In [28]:
GPperCat = data.merge(grossProfitData.rename(columns = {'Dish': 'item'}), on = 'item', how = 'left')[['item', 
                                                                                           'qty', 'category', 'Gross Profit']]

GPperCat.loc[:, 'Gross Profit'] = GPperCat['Gross Profit'].str.replace('$',"").astype(float)

In [29]:
GPperCat.dropna(inplace = True)

In [30]:
GPperCat = GPperCat.groupby('category').sum().reset_index()[['category', 'qty', 'Gross Profit']]
GPperCat['GPpercentBreakdown'] = round((GPperCat['Gross Profit'] / GPperCat['Gross Profit'].sum()) * 100, 2)

In [31]:
GPperCat.sort_values(by = 'GPpercentBreakdown', ascending = False)

,category,qty,Gross Profit,GPpercentBreakdown
7,pasta,1881,25287.08,31.93
8,pizza,1696,20494.12,25.88
1,fromTheLand,659,9053.54,11.43
13,starters,704,4533.13,5.72
0,additional,910,4172.00,5.27
11,softDrinks,952,3552.00,4.48
5,gourmetPizza,202,3398.53,4.29
2,fromTheSea,88,2188.18,2.76
10,salads,226,2166.48,2.74
9,risotto,153,2121.27,2.68


The above dataframe shows the weighted average % gross profit per category. We can see that Pasta, Pizza, From The Land, and Starters have the top most Gross Profit percentages among all the categories.

**Note** Additional category includes those items which are not listed in the Gross Profit excel sheet (majority of the items are from the Wine, Beers, and Liquors category.

##### **3. Rank the popularity of items in each category. (Arrabiata (in Pasta) is 10% of pasta sales, Broccoli is 5% of pasta sales). What is gross profit per each dish?**

In [32]:
dataWithGP = data.merge(grossProfitData.rename(columns = {'Dish': 'item'}), 
                        on = 'item', how = 'left')[['item', 'qty', 'price', 'date', 'order_id', 'category', 'Gross Profit']]

dataWithGP.loc[:, 'Gross Profit'] = dataWithGP['Gross Profit'].str.replace('$',"").astype(float)
dataWithGP.dropna(inplace = True)

In [33]:
dataWithGP

,item,qty,price,date,order_id,category,Gross Profit
0,Gelati Cone,1,5.0,27/01/2022,124241,gelati,3.28
1,Chicken & Mushroom Main,1,21.5,14/01/2022,123672,pasta,14.18
2,Chicken & Mushroom Main,1,21.5,14/01/2022,123672,pasta,16.95
3,Coke Zero,1,4.5,14/01/2022,123672,softDrinks,4.50
4,Lemon Lime Bitters,1,5.0,14/01/2022,123672,softDrinks,5.00
...,...,...,...,...,...,...,...
7811,Supremo Small,1,19.5,2022-05-04,127371,gourmetPizza,16.91
7812,Wine Corkage,1,7.0,2022-05-04,127371,additional,7.00
7813,Antipasto Small,1,19.5,2022-05-04,127374,gourmetPizza,14.74
7814,Margherita Small,1,12.5,2022-05-04,127376,pizza,8.89


## Popularity of Items in Each Category

#### Pasta

The following Table shows the popularity of items in this category and the gross profit per dish in this category

In [34]:
itemQty = dataWithGP[dataWithGP['category'] == 'pasta'].groupby('item').sum().reset_index()

itemQty['PopularityPercentBreakdown'] = round((itemQty['qty'] / itemQty['qty'].sum()) * 100, 2)
itemQty['GPpercentBreakdown'] = round((itemQty['Gross Profit'] / itemQty['Gross Profit'].sum()) * 100, 2)

itemQty.sort_values(by = ['GPpercentBreakdown'], ascending = False)[['item', 'PopularityPercentBreakdown', 
                                                                     'GPpercentBreakdown']]

,item,PopularityPercentBreakdown,GPpercentBreakdown
8,Chicken & Mushroom Main,8.19,8.86
2,Bolognese Entree,8.93,7.88
7,Carbonara Main,6.65,7.29
6,Carbonara Entree,7.87,6.98
19,Romana Entree,7.60,6.79
20,Romana Main,5.64,6.57
3,Bolognese Main,6.06,6.15
11,Garlic Prawns Entree,6.22,6.12
0,Arrabiata Entree,6.49,5.68
16,Marinara Main,4.20,5.53


#### Soft Drinks

The following Table shows the popularity of items in this category and the gross profit per dish in this category

In [36]:
itemQty = dataWithGP[dataWithGP['category'] == 'softDrinks'].groupby('item').sum().reset_index()

itemQty['PopularityPercentBreakdown'] = round((itemQty['qty'] / itemQty['qty'].sum()) * 100, 2)
itemQty['GPpercentBreakdown'] = round((itemQty['Gross Profit'] / itemQty['Gross Profit'].sum()) * 100, 2)

itemQty.sort_values(by = ['GPpercentBreakdown'], ascending = False)[['item', 'PopularityPercentBreakdown', 
                                                                     'GPpercentBreakdown']]

,item,PopularityPercentBreakdown,GPpercentBreakdown
0,Coke,34.98,31.04
2,Lemon Lime Bitters,26.16,27.31
1,Coke Zero,18.28,16.47
3,Mineral Water 500Ml,8.40,13.40
4,Sprite,12.18,11.78


#### From The Land

The following Table shows the popularity of items in this category and the gross profit per dish in this category

In [37]:
itemQty = dataWithGP[dataWithGP['category'] == 'fromTheLand'].groupby('item').sum().reset_index()

itemQty['PopularityPercentBreakdown'] = round((itemQty['qty'] / itemQty['qty'].sum()) * 100, 2)
itemQty['GPpercentBreakdown'] = round((itemQty['Gross Profit'] / itemQty['Gross Profit'].sum()) * 100, 2)

itemQty.sort_values(by = ['GPpercentBreakdown'], ascending = False)[['item', 'PopularityPercentBreakdown', 
                                                                     'GPpercentBreakdown']]

,item,PopularityPercentBreakdown,GPpercentBreakdown
1,Parma Chicken Main,17.00,20.20
0,Parma Chicken Entree,13.96,12.76
5,Schnitzel Chicken Entree,14.26,12.65
3,Parma Veal Main,10.93,12.32
4,Scallopine,14.87,11.37
8,Schnitzel Veal Main,6.07,7.51
2,Parma Veal Entree,8.04,7.23
7,Schnitzel Veal Entree,7.13,6.77
6,Schnitzel Chicken Main,5.77,6.72
9,Vitello Sorrento,1.97,2.47


#### Starters

The following Table shows the popularity of items in this category and the gross profit per dish in this category

In [38]:
itemQty = dataWithGP[dataWithGP['category'] == 'starters'].groupby('item').sum().reset_index()

itemQty['PopularityPercentBreakdown'] = round((itemQty['qty'] / itemQty['qty'].sum()) * 100, 2)
itemQty['GPpercentBreakdown'] = round((itemQty['Gross Profit'] / itemQty['Gross Profit'].sum()) * 100, 2)

itemQty.sort_values(by = ['GPpercentBreakdown'], ascending = False)[['item', 'PopularityPercentBreakdown', 
                                                                     'GPpercentBreakdown']]

,item,PopularityPercentBreakdown,GPpercentBreakdown
4,Garlic Bread,54.97,53.43
1,Bowl Of Chips,29.40,18.69
3,Bruschetta Tomato,9.09,14.59
5,Oysters Kilpatrick Dozen,2.70,7.43
2,Bruschetta Capsicum,2.27,3.43
0,Affetato Platter,1.56,2.44


#### Pancakes

The following Table shows the popularity of items in this category and the gross profit per dish in this category

In [42]:
itemQty = dataWithGP[dataWithGP['category'] == 'pancakes'].groupby('item').sum().reset_index()

itemQty['PopularityPercentBreakdown'] = round((itemQty['qty'] / itemQty['qty'].sum()) * 100, 2)
itemQty['GPpercentBreakdown'] = round((itemQty['Gross Profit'] / itemQty['Gross Profit'].sum()) * 100, 2)

itemQty.sort_values(by = ['GPpercentBreakdown'], ascending = False)[['item', 'PopularityPercentBreakdown', 
                                                                     'GPpercentBreakdown']]

,item,PopularityPercentBreakdown,GPpercentBreakdown
2,Jamaican Pancake,41.56,42.36
1,Citrus Pancake,23.38,23.32
3,Plain Pancake,23.38,20.57
0,Blueberry Pancake,11.69,13.75


#### Salads

The following Table shows the popularity of items in this category and the gross profit per dish in this category

In [43]:
itemQty = dataWithGP[dataWithGP['category'] == 'salads'].groupby('item').sum().reset_index()

itemQty['PopularityPercentBreakdown'] = round((itemQty['qty'] / itemQty['qty'].sum()) * 100, 2)
itemQty['GPpercentBreakdown'] = round((itemQty['Gross Profit'] / itemQty['Gross Profit'].sum()) * 100, 2)

itemQty.sort_values(by = ['GPpercentBreakdown'], ascending = False)[['item', 'PopularityPercentBreakdown', 
                                                                     'GPpercentBreakdown']]

,item,PopularityPercentBreakdown,GPpercentBreakdown
3,Warm Beef Salad,14.16,28.19
2,Side Salad,53.10,24.58
0,Caesar Salad,11.95,16.71
4,Warm Chicken Salad,10.62,16.16
1,Seafood Salad,10.18,14.37


##### **4. Record the average check size per month**

In [49]:
data['date'] = pd.to_datetime(data['date'])

In [50]:
data['month'] = data['date'].dt.month

In [51]:
avgCheck1 = data[data['month'] == 1]['qty'].sum() / len(data[data['month'] == 1]['order_id'].unique())
avgCheck2 = data[data['month'] == 2]['qty'].sum() / len(data[data['month'] == 1]['order_id'].unique())
avgCheck3 = data[data['month'] == 3]['qty'].sum() / len(data[data['month'] == 1]['order_id'].unique())
avgCheck4 = data[data['month'] == 4]['qty'].sum() / len(data[data['month'] == 1]['order_id'].unique())
avgCheck5 = data[data['month'] == 5]['qty'].sum() / len(data[data['month'] == 1]['order_id'].unique())

In [52]:
print('Average Check Size for Month 1:', round(avgCheck1,2))
print('Average Check Size for Month 2:', round(avgCheck2,2))
print('Average Check Size for Month 3:', round(avgCheck3,2))
print('Average Check Size for Month 4:', round(avgCheck4,2))
print('Average Check Size for Month 5:', round(avgCheck5,2))

Average Check Size for Month 1: 3.97
Average Check Size for Month 2: 5.41
Average Check Size for Month 3: 5.36
Average Check Size for Month 4: 6.06
Average Check Size for Month 5: 0.83
